In [1]:
import sqlite3
import pandas as pd

In [13]:
c = cnx.cursor()

In [18]:
c.execute('''SELECT * FROM sqlite_master''')

In [19]:
c.fetchall()

[('table',
  'sqlite_sequence',
  'sqlite_sequence',
  4,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'Player_Attributes',
  'Player_Attributes',
  11,
  'CREATE TABLE "Player_Attributes" (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`player_fifa_api_id`\tINTEGER,\n\t`player_api_id`\tINTEGER,\n\t`date`\tTEXT,\n\t`overall_rating`\tINTEGER,\n\t`potential`\tINTEGER,\n\t`preferred_foot`\tTEXT,\n\t`attacking_work_rate`\tTEXT,\n\t`defensive_work_rate`\tTEXT,\n\t`crossing`\tINTEGER,\n\t`finishing`\tINTEGER,\n\t`heading_accuracy`\tINTEGER,\n\t`short_passing`\tINTEGER,\n\t`volleys`\tINTEGER,\n\t`dribbling`\tINTEGER,\n\t`curve`\tINTEGER,\n\t`free_kick_accuracy`\tINTEGER,\n\t`long_passing`\tINTEGER,\n\t`ball_control`\tINTEGER,\n\t`acceleration`\tINTEGER,\n\t`sprint_speed`\tINTEGER,\n\t`agility`\tINTEGER,\n\t`reactions`\tINTEGER,\n\t`balance`\tINTEGER,\n\t`shot_power`\tINTEGER,\n\t`jumping`\tINTEGER,\n\t`stamina`\tINTEGER,\n\t`strength`\tINTEGER,\n\t`long_shots`\tINTEGER,\n\t`aggr

## Problem 1

Which team scored the most points when playing at home?



In [2]:
cnx = sqlite3.connect('database.sqlite')

In [3]:
pd.read_sql_query('''SELECT team_long_name AS Team_Name, team_short_name AS Team_Abv, SUM(home_team_goal) AS Home_Goals
FROM Match
JOIN Team ON Team.team_api_id = Match.home_team_api_id
GROUP BY Match.home_team_api_id
ORDER BY Home_Goals DESC
LIMIT 10''', cnx)

,Team_Name,Team_Abv,Home_Goals
0,Real Madrid CF,REA,505
1,FC Barcelona,BAR,495
2,Celtic,CEL,389
3,FC Bayern Munich,BMU,382
4,PSV,PSV,370
5,Manchester City,MCI,365
6,Ajax,AJA,360
7,FC Basel,BAS,344
8,Manchester United,MUN,338
9,Chelsea,CHE,333


## Problem 2

Did this team also score the most points when playing away?

In [4]:
pd.read_sql_query('''SELECT team_long_name AS Team_Name, team_short_name AS Team_Abv, SUM(away_team_goal) AS Away_Goals
FROM Match
JOIN Team ON Team.team_api_id = Match.away_team_api_id
GROUP BY Match.away_team_api_id
ORDER BY Away_Goals DESC
LIMIT 10''', cnx)

,Team_Name,Team_Abv,Away_Goals
0,FC Barcelona,BAR,354
1,Real Madrid CF,REA,338
2,Celtic,CEL,306
3,Ajax,AJA,287
4,PSV,PSV,282
5,FC Basel,BAS,275
6,FC Bayern Munich,BMU,271
7,Arsenal,ARS,267
8,Borussia Dortmund,DOR,253
9,Chelsea,CHE,250


## Problem 3

How many matches resulted in a tie?

In [5]:
pd.read_sql_query('''SELECT COUNT(match_api_id) AS Ties
FROM Match
WHERE away_team_goal = home_team_goal
LIMIT 10''', cnx)

,Ties
0,6596


## Problem 4

How many players have Smith for their last name? How many have 'smith' anywhere in their name?



In [6]:
pd.read_sql_query('''SELECT COUNT(player_name) FROM Player 
WHERE player_name LIKE '% SMITH'
''', cnx)

,COUNT(player_name)
0,15


In [7]:
pd.read_sql_query('''SELECT COUNT(player_name) FROM Player 
WHERE player_name LIKE '%SMITH%'
''', cnx)

,COUNT(player_name)
0,18


## Problem 5

What was the median tie score? Use the value determined in the previous question for the number of tie games. Hint: PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the WITH command to store stepwise results as a table and then operate on these results.

In [31]:
pd.read_sql_query('''WITH Ties AS (SELECT COUNT(match_api_id) AS Num_Ties
FROM Match
WHERE away_team_goal = home_team_goal)
SELECT away_team_goal as Away_Goals, home_team_goal AS Home_Goals, Num_Ties
FROM Match, Ties
WHERE away_team_goal = home_team_goal
ORDER BY Home_Goals DESC
LIMIT 1 OFFSET 6596/2
''', cnx)

,Away_Goals,Home_Goals,Num_Ties
0,1,1,6596


In [32]:
df = pd.read_sql_query('''WITH Ties AS (SELECT COUNT(match_api_id) AS Num_Ties
FROM Match
WHERE away_team_goal = home_team_goal)
SELECT away_team_goal as Away_Goals, home_team_goal AS Home_Goals, Num_Ties
FROM Match, Ties
WHERE away_team_goal = home_team_goal
ORDER BY Home_Goals DESC
''', cnx)

In [33]:
df['Away_Goals'].median()

1.0

## Problem 6

What percentage of players prefer their left or right foot? Hint: Calculate either the right or left foot, whichever is easier based on how you setup the problem.



In [9]:
right_db = pd.read_sql_query('''
SELECT COUNT(player_api_id)
FROM Player_Attributes
WHERE preferred_foot = 'right'
LIMIT 1
''', cnx)
right = int(right_db.iloc[0])

In [10]:
total_db = pd.read_sql_query('''SELECT COUNT(*) FROM Player_Attributes LIMIT 1''',cnx)
total = int(total_db.iloc[0])

In [11]:
right/total

0.7523127765276283